In [229]:
!pip install pyvis
!pip install networkx

In [230]:
import pandas as pd 
import numpy as np
import seaborn as sns

from pyvis.network import Network 
import networkx as nx

In [231]:
df = pd.read_csv("origin_destination_train_202302.csv")
df1 = df.loc[df.DAY_TYPE == "WEEKDAY"]
df2 = df.loc[df.DAY_TYPE == "WEEKENDS/HOLIDAY"]

df1 = df

In [232]:
df3 = pd.read_csv("weekday_passenger_vol.csv").rename(columns={"PT_CODE":"ORIGIN_PT_CODE"})
df4 = pd.read_csv("weekends_passenger_vol.csv").rename(columns={"PT_CODE":"ORIGIN_PT_CODE"})

In [233]:
df3 = df3.append(df4)

In [234]:
#df1 = df1.merge(df3[["ORIGIN_PT_CODE","TIME_PER_HOUR", "DAY_TYPE",'TOTAL_TAP_IN_VOLUME']], on=["ORIGIN_PT_CODE","TIME_PER_HOUR", "DAY_TYPE"])

In [235]:
df1.loc[df1.ORIGIN_PT_CODE.str.contains("B*/", regex=True)]

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
20,2023-02,WEEKENDS/HOLIDAY,7,TRAIN,NE7/DT12,DT7,56
21,2023-02,WEEKDAY,7,TRAIN,NE7/DT12,DT7,309
35,2023-02,WEEKDAY,9,TRAIN,EW21/CC22,NE9,29
38,2023-02,WEEKENDS/HOLIDAY,9,TRAIN,EW21/CC22,NE9,4
40,2023-02,WEEKDAY,7,TRAIN,CC17/TE9,TE16,67
...,...,...,...,...,...,...,...
729305,2023-02,WEEKENDS/HOLIDAY,0,TRAIN,NE7/DT12,NE16/STC,6
729308,2023-02,WEEKDAY,10,TRAIN,NS21/DT11,NS20,2234
729309,2023-02,WEEKENDS/HOLIDAY,10,TRAIN,NS21/DT11,NS20,1101
729319,2023-02,WEEKENDS/HOLIDAY,13,TRAIN,NS4/BP1,NE8,28


In [236]:
import re

In [237]:
reg = r"(/B.*)|(^B.*/)|(/S.*)|(^S.*/)|(/P.*)|(^P.*/)|(/P.*)|(P.*/)"

In [238]:
reg = r"(/B.*|^B.*/|/S.*|^S.*/|/P.*|^P.*/)"

In [239]:
re.search(reg, "BP1/NS4")

<re.Match object; span=(0, 4), match='BP1/'>

In [240]:
df1['ORIGIN_PT_CODE'] = df1.ORIGIN_PT_CODE.str.replace(reg, "")
df3['ORIGIN_PT_CODE'] = df3.ORIGIN_PT_CODE.str.replace(reg, "")

df1['DESTINATION_PT_CODE'] = df1.DESTINATION_PT_CODE.str.replace(reg, "")

C:\Users\leish\AppData\Local\Temp/ipykernel_20924/685950100.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['ORIGIN_PT_CODE'] = df1.ORIGIN_PT_CODE.str.replace(reg, "")
C:\Users\leish\AppData\Local\Temp/ipykernel_20924/685950100.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df3['ORIGIN_PT_CODE'] = df3.ORIGIN_PT_CODE.str.replace(reg, "")
C:\Users\leish\AppData\Local\Temp/ipykernel_20924/685950100.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['DESTINATIoN_PT_CODE'] = df1.DESTINATION_PT_CODE.str.replace(reg, "")


In [241]:
df1 = df1.drop(df1.loc[(df1.ORIGIN_PT_CODE.str.startswith("B")) | (df1.ORIGIN_PT_CODE.str.startswith("P"))| (df1.ORIGIN_PT_CODE.str.startswith("S")) |(df1.DESTINATION_PT_CODE.str.startswith("B")) | (df1.DESTINATION_PT_CODE.str.startswith("P"))| (df1.DESTINATION_PT_CODE.str.startswith("S"))].index)

In [242]:
df3 = df3.drop(df3.loc[(df3.ORIGIN_PT_CODE.str.startswith("B")) | (df3.ORIGIN_PT_CODE.str.startswith("P"))| (df3.ORIGIN_PT_CODE.str.startswith("S"))].index)

In [243]:
df3.to_csv("passenger_vol.csv")

In [244]:
df3

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME
0,2023-02,WEEKDAY,0,TRAIN,DT10/TE11,34,162
1,2023-02,WEEKDAY,0,TRAIN,NS27/CE2/TE20,20,85
2,2023-02,WEEKDAY,0,TRAIN,EW19,31,309
3,2023-02,WEEKDAY,0,TRAIN,NS8,375,1697
4,2023-02,WEEKDAY,0,TRAIN,NE16,280,2135
...,...,...,...,...,...,...,...
3400,2023-02,WEEKENDS/HOLIDAY,23,TRAIN,NS27/CE2/TE20,185,115
3406,2023-02,WEEKENDS/HOLIDAY,23,TRAIN,NE7/DT12,1967,1430
3408,2023-02,WEEKENDS/HOLIDAY,23,TRAIN,DT29,202,1300
3409,2023-02,WEEKENDS/HOLIDAY,23,TRAIN,TE4,195,287


In [245]:
df1 = df1.sort_values(by=['TIME_PER_HOUR','DESTINATION_PT_CODE'])

In [246]:
total_trips = df1.groupby(["ORIGIN_PT_CODE", "TIME_PER_HOUR", "DAY_TYPE"]).sum().reset_index().rename({"TOTAL_TRIPS":"TOTAL_TRIPS_AGG"},axis=1)

In [247]:
df1 = pd.merge(df1, total_trips, on=["ORIGIN_PT_CODE","TIME_PER_HOUR", "DAY_TYPE"])

In [248]:
df1['PROB_TRIP'] = df1.apply(lambda x: round(x['TOTAL_TRIPS']/x['TOTAL_TRIPS_AGG'],3),axis=1)

In [ ]:
df1['CUMSUM'] = df1.groupby(['ORIGIN_PT_CODE','TIME_PER_HOUR','DAY_TYPE'])['PROB_TRIP'].cumsum()


In [ ]:
171 * 171

29241

In [ ]:
df1.DESTINATION_PT_CODE.unique()

array(['DT1', 'CC12', 'CC16', 'EW24/NS1', 'NS10', 'NS11', 'NS13', 'NS4',
       'NS5', 'NS7', 'NS9/TE2', 'TE1', 'TE3', 'TE4', 'TE5', 'TE6', 'TE7',
       'CC10/DT26', 'DT10/TE11', 'DT18', 'DT21', 'DT22', 'DT25', 'DT27',
       'DT28', 'DT29', 'DT3', 'DT30', 'DT31', 'DT33', 'DT34', 'DT5',
       'DT7', 'EW12/DT14', 'EW2/DT32', 'NE4/DT19', 'NE7/DT12',
       'NS21/DT11', 'CC19/DT9', 'CC4/DT15', 'DT24', 'DT6', 'DT2', 'EW20',
       'EW8/CC9', 'NS17/CC15', 'CC11', 'NE12/CC13', 'NS18', 'CG1/DT35',
       'EW3', 'EW4', 'NE14', 'CC3', 'CC7', 'CC8', 'DT13', 'DT17', 'DT20',
       'EW1', 'EW10', 'EW11', 'EW5', 'EW6', 'EW7', 'EW9', 'CC17/TE9',
       'CC23', 'CC5', 'CE1/DT16', 'DT8', 'CC21', 'NS12', 'NS14', 'NS15',
       'NS16', 'NS19', 'NS20', 'NS8', 'NS2', 'NS23', 'NS3', 'CC14',
       'CC25', 'CC26', 'EW21/CC22', 'EW27', 'NS24/NE6/CC1', 'TE14/NS22',
       'TE8', 'EW23', 'NE10', 'NE11', 'NE13', 'NE15', 'NE16', 'NE17',
       'NE8', 'NE9', 'CC24', 'DT23', 'CC20', 'CC27', 'EW22', 'EW25',
     

In [ ]:
df1.to_csv("master_prob_trip.csv")

In [ ]:
df1 = df1.drop(['YEAR_MONTH','PT_TYPE','PROB_TRIP','TOTAL_TRIPS_AGG','TOTAL_TRIPS'], axis=1)

In [ ]:
df1.to_csv("prob_trip.csv")

In [ ]:
df1['ORIGIN_PT_CODE'] = df1.ORIGIN_PT_CODE.str.replace(reg, "")
df1['DESTINATION_PT_CODE'] = df1.DESTINATION_PT_CODE.str.replace(reg, "")

C:\Users\leish\AppData\Local\Temp/ipykernel_20924/1506593578.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['ORIGIN_PT_CODE'] = df1.ORIGIN_PT_CODE.str.replace(reg, "")
C:\Users\leish\AppData\Local\Temp/ipykernel_20924/1506593578.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['DESTINATION_PT_CODE'] = df1.DESTINATION_PT_CODE.str.replace(reg, "")


In [ ]:
df1 = df1.drop(df1.loc[(df1.ORIGIN_PT_CODE.str.startswith("B")) | (df1.ORIGIN_PT_CODE.str.startswith("P"))| (df1.ORIGIN_PT_CODE.str.startswith("S"))].index)
df1 = df1.drop(df1.loc[(df1.DESTINATION_PT_CODE.str.startswith("B")) | (df1.DESTINATION_PT_CODE.str.startswith("P"))| (df1.DESTINATION_PT_CODE.str.startswith("S"))].index)

In [ ]:
cost_df = df1.groupby(['ORIGIN_PT_CODE','DESTINATION_PT_CODE']).mean().reset_index()
cost_df = cost_df.pivot(index="ORIGIN_PT_CODE", columns="DESTINATION_PT_CODE", values="CUMSUM")

In [ ]:
cost_df.columns[cost_df.columns.str.contains("EW16")]

Index(['EW16/NE3/TE17'], dtype='object', name='DESTINATION_PT_CODE')

In [ ]:
for x,y in zip(cost_df.columns, cost_df.index):
    if x != y:
        print(x,y)

In [ ]:
cost_df.index

Index(['CC10/DT26', 'CC11', 'CC12', 'CC14', 'CC16', 'CC17/TE9', 'CC19/DT9',
       'CC2', 'CC20', 'CC21',
       ...
       'TE16', 'TE18', 'TE19', 'TE22', 'TE3', 'TE4', 'TE5', 'TE6', 'TE7',
       'TE8'],
      dtype='object', name='ORIGIN_PT_CODE', length=134)

In [ ]:
length = len(df1['ORIGIN_PT_CODE'].unique())
stations = df1['ORIGIN_PT_CODE'].unique()

In [ ]:
import math
cost = [[math.inf] * length for k in range(length)]

In [ ]:
cost_df.index[0], cost_df.columns[1]

('CC10/DT26', 'CC11')

In [ ]:
a = [1,2,3]
b = [4,5,6]

for x,y in zip(a,b):
    print(x,y)

1 4
2 5
3 6


In [ ]:
def isNextId(row1, row2):
    row1s = row1.split("/")
    row2s = row2.split("/")
    for x in row1s:
        for y in row2s:
            if x[:2] == y[:2]:
                number = int(x[2:]) + 1
                steps = 1
                while steps < 3:
                    if int(y[2:]) == number:
                        return True 
                    
                    elif len(cost_df.columns[cost_df.columns.str.contains(x[:2] + f"{number:1}")]) >= 1:
                        return False
                    number += 1
                    steps += 1
        
    return False

In [ ]:
def isPrevId(row1, row2):
    row1s = row1.split("/")
    row2s = row2.split("/")

    for x in row1s:
        for y in row2s:
            if x[:2] == y[:2]:
                number = int(x[2:]) - 1 
                steps = -1
                while number > 0 and steps < 3:
                    if int(y[2:]) == number:
                        return True 
                    elif len(cost_df.columns[cost_df.columns.str.contains(x[:2] + f"{number:1}")]) >= 1:
                        return False
                    number -= 1
                    steps -= 1
        
    return False

In [ ]:
pi = [[0] * length for i in range(length)]

In [ ]:
for i in range(length):
    for j in range(length):
        if i == j:
            cost[i][j] = 0
        elif isNextId(cost_df.index[i],cost_df.columns[j]):
            cost[i][j] = 1
            pi[i][j] = i
        elif isPrevId(cost_df.index[i], cost_df.columns[j]):
            cost[i][j] = 1
            pi[i][j] = i

In [ ]:
df_cost = pd.DataFrame(cost, index=cost_df.index, columns=cost_df.columns)
df_cost

DESTINATION_PT_CODE,CC10/DT26,CC11,CC12,CC14,CC16,CC17/TE9,CC19/DT9,CC2,CC20,CC21,...,TE16,TE18,TE19,TE22,TE3,TE4,TE5,TE6,TE7,TE8
ORIGIN_PT_CODE,,,,,,,,,,,,,,,,,,,,,
CC10/DT26,0.0,1.0,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
CC11,1.0,0.0,1.0,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
CC12,inf,1.0,0.0,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
CC14,inf,inf,inf,0.0,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
CC16,inf,inf,inf,inf,0.0,1.0,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TE4,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,1.0,0.0,1.0,inf,inf,inf
TE5,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,1.0,0.0,1.0,inf,inf
TE6,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,1.0,0.0,1.0,inf


In [ ]:
pd.DataFrame(pi, index=cost_df.index, columns=cost_df.columns)

DESTINATION_PT_CODE,CC10/DT26,CC11,CC12,CC14,CC16,CC17/TE9,CC19/DT9,CC2,CC20,CC21,...,TE16,TE18,TE19,TE22,TE3,TE4,TE5,TE6,TE7,TE8
ORIGIN_PT_CODE,,,,,,,,,,,,,,,,,,,,,
CC10/DT26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CC11,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CC12,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CC14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CC16,0,0,0,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TE4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,129,0,129,0,0,0
TE5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,130,0,130,0,0
TE6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,131,0,131,0


In [ ]:
for k in range(length):
    for i in range(length):
        for j in range(length): 
            if cost[i][k] + cost[k][j] < cost[i][j]:
                cost[i][j] = cost[i][k] + cost[k][j]
                pi[i][j] = pi[k][j]

In [ ]:
df_cost = pd.DataFrame(cost, index=cost_df.index, columns=cost_df.columns)
df_cost

DESTINATION_PT_CODE,CC10/DT26,CC11,CC12,CC14,CC16,CC17/TE9,CC19/DT9,CC2,CC20,CC21,...,TE16,TE18,TE19,TE22,TE3,TE4,TE5,TE6,TE7,TE8
ORIGIN_PT_CODE,,,,,,,,,,,,,,,,,,,,,
CC10/DT26,0,1,2,4,6,7,8,8,9,10,...,9,9,9,9,13,12,11,10,9,8
CC11,1,0,1,3,5,6,7,9,8,9,...,10,10,10,10,12,11,10,9,8,7
CC12,2,1,0,2,4,5,6,8,7,8,...,10,11,11,11,11,10,9,8,7,6
CC14,4,3,2,0,2,3,4,8,5,6,...,8,10,11,11,9,8,7,6,5,4
CC16,6,5,4,2,0,1,2,6,3,4,...,6,8,9,9,7,6,5,4,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TE4,12,11,10,8,6,5,6,10,7,8,...,10,12,13,13,1,0,1,2,3,4
TE5,11,10,9,7,5,4,5,9,6,7,...,9,11,12,12,2,1,0,1,2,3
TE6,10,9,8,6,4,3,4,8,5,6,...,8,10,11,11,3,2,1,0,1,2


In [ ]:
pi_df = pd.DataFrame(pi, index=cost_df.index, columns=cost_df.columns)
pi_df.to_csv("journey_times.csv")

In [ ]:
(pi_df.columns == "NS4/BP1").sum()

0

In [ ]:
df.in

SyntaxError: invalid syntax (Temp/ipykernel_20924/2188923685.py, line 1)

In [ ]:
def get_shortest_path(one,two):
    if one != two:
        get_shortest_path(one, pi[one][two])
    print(cost_df.index[two])

In [ ]:
get_shortest_path(21, 100)

CC8
EW8/CC9
CC10/DT26
CC11
CC12
NE12/CC13
CC14
NS17/CC15
NS16


In [ ]:
cost_df.index[21], cost_df.index[100]

('CC8', 'NS16')

In [ ]:
pd.DataFrame(pi, index=cost_df.index, columns=cost_df.columns)

DESTINATION_PT_CODE,CC10/DT26,CC11,CC12,CC14,CC16,CC17/TE9,CC19/DT9,CC2,CC20,CC21,...,TE16,TE18,TE19,TE22,TE3,TE4,TE5,TE6,TE7,TE8
ORIGIN_PT_CODE,,,,,,,,,,,,,,,,,,,,,
CC10/DT26,0,0,1,83,101,4,5,16,6,8,...,54,54,110,110,128,129,130,131,132,5
CC11,1,0,1,83,101,4,5,16,6,8,...,54,54,110,110,128,129,130,131,132,5
CC12,1,2,0,83,101,4,5,108,6,8,...,122,54,110,110,128,129,130,131,132,5
CC14,1,2,83,0,101,4,5,108,6,8,...,122,54,110,110,128,129,130,131,132,5
CC16,1,2,83,101,0,4,5,108,6,8,...,122,54,110,110,128,129,130,131,132,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TE4,1,2,83,101,5,132,5,108,6,8,...,122,54,110,110,128,0,128,129,130,131
TE5,1,2,83,101,5,132,5,108,6,8,...,122,54,110,110,128,129,0,129,130,131
TE6,1,2,83,101,5,132,5,108,6,8,...,122,54,110,110,128,129,130,0,130,131


In [ ]:
pd.DataFrame(cost, index=cost_df.index, columns=cost_df.columns)

DESTINATION_PT_CODE,CC10/DT26,CC11,CC12,CC14,CC16,CC17/TE9,CC19/DT9,CC2,CC20,CC21,...,TE16,TE18,TE19,TE22,TE3,TE4,TE5,TE6,TE7,TE8
ORIGIN_PT_CODE,,,,,,,,,,,,,,,,,,,,,
CC10/DT26,0,1,2,4,6,7,8,8,9,10,...,9,9,9,9,13,12,11,10,9,8
CC11,1,0,1,3,5,6,7,9,8,9,...,10,10,10,10,12,11,10,9,8,7
CC12,2,1,0,2,4,5,6,8,7,8,...,10,11,11,11,11,10,9,8,7,6
CC14,4,3,2,0,2,3,4,8,5,6,...,8,10,11,11,9,8,7,6,5,4
CC16,6,5,4,2,0,1,2,6,3,4,...,6,8,9,9,7,6,5,4,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TE4,12,11,10,8,6,5,6,10,7,8,...,10,12,13,13,1,0,1,2,3,4
TE5,11,10,9,7,5,4,5,9,6,7,...,9,11,12,12,2,1,0,1,2,3
TE6,10,9,8,6,4,3,4,8,5,6,...,8,10,11,11,3,2,1,0,1,2


In [ ]:
df1.groupby(["ORIGIN_PT_CODE", "TIME_PER_HOUR", "DAY_TYPE"]).max().reset_index().sort_values(by="CUMSUM")

,ORIGIN_PT_CODE,TIME_PER_HOUR,DAY_TYPE,DESTINATION_PT_CODE,CUMSUM
3510,NE17/PTC,0,WEEKDAY,NE10,0.005
3470,NE16/STC,0,WEEKDAY,NE17/PTC,0.059
3471,NE16/STC,0,WEEKENDS/HOLIDAY,NE17/PTC,0.109
3431,NE15,0,WEEKENDS/HOLIDAY,NE17/PTC,0.352
3430,NE15,0,WEEKDAY,NE17/PTC,0.409
...,...,...,...,...,...
617,CC28,13,WEEKDAY,TE8,1.028
1763,DT5,7,WEEKENDS/HOLIDAY,TE8,1.029
3291,NE11,14,WEEKENDS/HOLIDAY,TE8,1.030
623,CC28,16,WEEKDAY,TE7,1.031


In [ ]:
df1.groupby(["ORIGIN_PT_CODE", "TIME_PER_HOUR"]).sum().reset_index()['TOTAL_TRIPS']

KeyError: 'TOTAL_TRIPS'

In [ ]:
df1